In [1]:
import pandas as pd

In [2]:
from gradelib import *

In [3]:
gp = GradingPipeline()

In [4]:
homeworks = LazyAssignments(lambda asmts: asmts.starting_with('home'))
labs = LazyAssignments(lambda asmts: asmts.starting_with('lab'))
projects = LazyAssignments(lambda asmts: asmts.starting_with('proj'))

In [5]:
@gp.step(0)
def load():
    return io.gradescope.read('./tests/examples/gradescope.csv')

In [6]:
@gp.step(1)
def preprocessing(gb: Gradebook):
    # set assignment groups
    gb.grading_groups = {
        'homeworks': (homeworks, 0.5),
        'labs': (normalize(labs(gb.assignments)), 0.25),
        'projects': (projects, 0.25)
    }

In [7]:
@gp.step(2)
def policies_(gb: Gradebook):
    # forgive 3 lates
    policies.penalize_lates(gb, forgive=1, within=homeworks + labs)
    
    # drop lowest homework
    policies.drop_lowest(gb, n=1, within=homeworks)
    
    # drop lowest lab
    policies.drop_lowest(gb, n=1, within=labs)

In [8]:
gb = gp.grade()

In [9]:
overview(gb)

Letter,A+,A,A-,B+,B,B-,C+,C,C-,D,F
Frequency,1,0,0,0,3,0,0,0,0,0,0


Loading BokehJS ...

,Frequency
Number of Lates,
1,1
2,2
5,1


,homeworks,labs,projects,overall score,letter,rank,percentile
<Fitzgerald Zelda>,0.991071,1.000000,0.901961,0.971026,A+,1,1.00
<Eldridge Justin>,0.794643,0.874322,0.882353,0.836490,B,2,0.75
<Eldridge Another>,0.794643,0.874322,0.882353,0.836490,B,3,0.50
<Obama Barack>,0.889423,0.625000,0.921569,0.831354,B,4,0.25


In [10]:
overview(gb, 'justin')

,homeworks,labs,projects,overall score,letter,rank,percentile
<Eldridge Justin>,0.794643,0.874322,0.882353,0.83649,B,2,0.75
